In [1]:
import requests
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas
import question
import random

/var/folders/g_/12fwrqfs2lgcf9rnql4m440dx849vl/T/ipykernel_66366/3108238445.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas


In [2]:
# Predicates
SUBCLASS_OF = "P279"
PART_OF = "P361"
INSTANCE_OF = "P31"

predicates = [SUBCLASS_OF, PART_OF, INSTANCE_OF]

In [3]:
wikidata_properties = {
    "named_after": "wdt:P138",
    "occupation": "wdt:P106",
    "has_use": "wdt:P366",
    "studied_in": "wdt:P2579",
    "subclass_of": "wdt:P279",
    "part_of": "wdt:P361",
    "field_of_work": "wdt:P101",
    "main_subject": "wdt:P921",
    "located_in_the_administrative_territorial_entity": "wdt:P131",
    "contains_administrative_territorial_entity": "wdt:P150",
    "practiced by": "wdt:P3095"
}



In [74]:
# Category Example: Team Sports
# Element Example: Baseball

class Generator:
    def __init__(self, sister_predicates):
        self.sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
        self.sister_predicates = sister_predicates
        self.check_answer_is_correct_pattern_num_runs = 0
    
    # SPARQL Helper Methods
    def run_query(self, query):
        '''
        Description:
            Takes in query and requests its output
        
        Arguments:
            query:string
        
        Returns:
            results:JSON
        '''
        # Set the query and the return format (JSON)
        self.sparql.setQuery(query)
        self.sparql.setReturnFormat(JSON)

        # Perform the query and convert the result to a Python dictionary
        results = self.sparql.query().convert()
        return results 
    def find_label_by_ID(self, ID):
        '''
        Description:
            Takes in QID and outputs its label
        
        Arguments:
            ID:string - "QXXXXX"
        
        Returns:
            label:string
        '''
        query = f'''
        SELECT ?itemLabel WHERE {{
            wd:{ID} rdfs:label ?itemLabel.
            FILTER(LANG(?itemLabel) = "en")
        }}
        '''
        results = self.run_query(query)
        return results["results"]["bindings"][0]["itemLabel"]["value"]
    def find_uri_by_label(self, label):
        '''
        Description:
            Takes in label and outputs its URI
        
        Arguments:
            label:string
        
        Returns:
            uri:string - 'http://www.wikidata.org/entity/XXXXXXX'
        '''
        
        # Create SPARQL query to find the URI for a given label
        query = f'''SELECT ?item WHERE {{ 
                    ?item rdfs:label "{label.replace('"', '\"')}"@en.
                    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }} 
                }} LIMIT 1'''
        
        try:
            results = self.run_query(query)

            # The first matching URI
            binding = results["results"]["bindings"][0]
            result = binding["item"]["value"]
            return result
        except Exception as e:
            print(f"An error occurred: {e}")
            return None
    def find_ID_by_label(self, label):
        '''
        Description:
            Takes in label and outputs its ID
        
        Arguments:
            label:string
        
        Returns:
            id:string - 'QXXXXXX'
        '''
        uri = self.find_uri_by_label(label)
        id = uri.split("/")[-1]
        return id
    def find_ID_by_uri(self, uri):
        '''
        Description:
            Takes in uri and outputs its ID
        
        Arguments:
            uri:string - http://www.wikidata.org/entity/QXXXXX or http://www.wikidata.org/prop/direct/PXXXXX 
        
        Returns:
            id:string - 'Q/PXXXXXX'
        '''
        return uri.split('/')[-1]
    def process_JSON(self, results_JSON, desired_variables):
        '''
        Description:
            Takes in output of a query and returns the desired outputs
        
        Arguments:
            results_JSON:JSON
            desired_variables:List(string) - variables wanted from output 
        
        Returns:
            results:List(ID)
        '''
        results = []
        for c in results_JSON['results']['bindings']:
            r = []
            for variable in desired_variables:
                r.append(self.find_ID_by_uri(c[variable]['value']))
            results.append(r)

        return results
    def random_items(self, items, count):
        '''
        Description:
            Takes in a list of items and outputs a random subset of them
        
        Arguments:
            items:list
            count:int
        
        Returns:
            selected:list() - List of count items
        '''
        
        selected = random.sample(items, count)
        return selected
    def generate_avoided_element_queries(self, avoided_elementIDs):
        '''
        Description:
            Takes in list of element IDs and outputs a string of SPARQL queries that avoid them
        
        Arguments:
            avoided_elementIDs:list(string)
        
        Returns:
            avoided_element_queries:string
        '''
        avoided_element_queries = ""
        for elementID in avoided_elementIDs:
            avoided_element_queries += f'wd:{elementID} ?predicates ?category.\n'
        return avoided_element_queries
    def sister_category(self, categoryID, avoided_elementIDs, included_elementIDs=[], n_items=3):
        '''
        Description:
            Takes in category and element IDs and outputs related categories that don't contain the element
        
        Arguments:
            categoryID:string
            avoided_elementIDs:list(string) - elements the category shouldn't cover
            included_elementIDs:list(string) - elements the category should cover - unimplemented
            n_items:int
        
        Returns:
            results:List(ID) - IDs of Sister categories of categoryID
            preds:List(ID) - IDS of predicates corresponding to results
            
        '''
        # Prepare elements
        avoided_element_queries = ""
        for elementID in avoided_elementIDs:
            avoided_element_queries += f'wd:{elementID} ?predicates ?category.\n'
        
        # included_element_queries = ""
        # if len(included_elementIDs) != 0:
        #     # included_element_queries += "VALUES ?includedElements {"
        #     for elementID in included_elementIDs:
        #         included_element_queries += f'wd:{elementID} '
        #     # included_element_queries += "}\n"
        #     # included_element_queries += "?includedElements ?predicates ?category.\n"
            


        # Make query
        desired_variables = ["category", "predicate"]
        query = f'''
                SELECT ?category ?categoryLabel ?predicate WHERE {{
                    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
                    VALUES ?predicates {{ {self.sister_predicates} }}
                    wd:{categoryID} ?predicates ?superCategory.
                    ?category ?predicates ?superCategory.

                    #VALUES ?includedElements {{ included_element_queries }}
                    #?includedElements ?predicates ?category.

                    FILTER(?category != wd:{categoryID})
                    FILTER NOT EXISTS {{
                        {avoided_element_queries}
                    }}
                
                    ?superCategory rdfs:label ?superCategoryLabel .
                    FILTER(LANG(?superCategoryLabel) = "en")
                    ?category rdfs:label ?categoryLabel .
                    FILTER(LANG(?categoryLabel) = "en")
                    BIND(?predicates AS ?predicate)
  
                }}
                LIMIT {n_items}
                #GROUP BY ?category ?categoryLabel
                #HAVING (COUNT(?includedElements) = {len(included_elementIDs)})
        '''
        
        results = self.run_query(query)
        
        results = self.process_JSON(results, desired_variables)
        return [r[0] for r in results], [r[1] for r in results]
    def sister_element(self, elementID, categoryID, exceptions=[], n_items=100):
        '''
        Description:
            Takes in element's ID and its category's ID and outputs related elements not in said category
        
        Arguments:
            elementID:string - "QXXXXX"
            categoryID:string - "QXXXXX"
            exceptions: list 
            n_items:int
        
        Returns:
            results:List(ID) - IDs of Sister topics of label
        '''
        
        #exeptions = [f'FILTER NOT EXISTS {{?item {exception} .}}\n' for exception in exceptions]
        exception_patterns = []
        for exception in exceptions:
            predicate, object = exception.split()
            pattern = f"""FILTER NOT EXISTS {{
                            ?sisterElement {predicate} ?exceptionCategory .
                            ?exceptionCategory (wdt:P279)* {object} .
                        }}\n"""
            exception_patterns.append(pattern)

        exceptions_string = "".join(exception_patterns)

        # Get Sister Catgeories
        sister_categories, sister_preds = self.sister_category(categoryID, [elementID], n_items=n_items)
        sister_preds = list(set(sister_preds))
        # Note: sister_preds[i] represents the predicate used to find sister_categories[i]
        '''
        Example:
        Some superCategory is a category that categoryID is within based on sister_preds[i]
        sister_categories[i] is in superCategory based on sister_preds[i]

        Thus, to find a sister of elementID (more like cousin), the sister_element must be within sister_categories[i] based on sister_preds[i]

        So, let's say the category is team sports.. Team sports is a SUBCLASS_OF sports.
        Animal sports is a SUBCLASS_OF sports
        Horse racing is a SUBCLASS_OF animal sports

        In that example:
        categoryID = team sports
        superCategory = sports
        sister_category[i] = animal sports
        sister_preds[i] = SUBCLASS_OF
        sister of elementID = horse racing

        Using sister_preds ensures that the sister_elements generated are closer to elementID
        '''

        #SPARQL query to find elements that are similar to the elementID but not in categoryID
        desired_variables = ["sisterElement"]
        query = f'''
        SELECT ?sisterElement ?sisterElementLabel WHERE {{
            VALUES ?predicates {{ {"".join([f'wdt:{p} ' for p in sister_preds])} }}
            VALUES ?sisterCategories {{ {"".join([f"wd:{sister} " for sister in sister_categories])} }}
            
            SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
            ?sisterElement ?predicates ?sisterCategories.
            ?sisterElement rdfs:label ?sisterElementLabel.

            FILTER(LANG(?sisterElementLabel) = "en")
            FILTER(?sisterElement != wd:{elementID})
            {exceptions_string}
            
        }}
        
        LIMIT {n_items}
        '''
        
        results = self.run_query(query)
        results = self.process_JSON(results, desired_variables)

        return [r[0] for r in results]
        
    
    # Printing Methods
    def display_as_table(self, results, n_items):
        '''
        Description:
            Creates a table of the n_items of the queried results
        
        Arguments:
            results:JSON - sister topics
            n_items:int
        
        Returns:
            None
        '''
        df = pandas.DataFrame.from_dict(results["results"]["bindings"][:n_items])
        df = df.applymap(lambda x: x["value"])
        pandas.set_option('display.max_rows', n_items) # n_items doesnt work here
        print(df)
    def print_question(self, question):
        '''
        Description:
            Prints the question
        
        Arguments:
            question:Question
        
        Returns:
            None
        '''
        try:
            print(f"Which of the following is a {self.find_label_by_ID(question.relation.predicate)} {self.find_label_by_ID(question.relation.object)}?")
        except:
            print(f"Which of the following is a {question.relation.predicate} {question.relation.object}?")
        print(question.all_answers)
        for index, answer in enumerate(question.all_answers):
            print(f"{index + 1}. {self.find_label_by_ID(answer)}")
    # def print_question_that_has_QIDs(self, question):
    #     print(f"Which of the following is a {self.find_label_by_ID(question.relation.predicate)} {self.find_label_by_ID(question.relation.object)}?")
    #     for index, answer in enumerate(question.all_answers):
    #         print(f"{index + 1}. {self.find_label_by_ID(answer)}")

    # Element-> Category+Elements Question Methods
    '''
    Which of the following is a subclass of Olympic sport?
    1. savika
    2. *baseball*
    3. memory sport
    4. snowshoe biathlon
    '''
    def get_category_relation(self, label):
        '''
        Description:
            Takes in label and outputs one of its categories in the form of a relation
            Relations are pairs of predicates and objects

        Arguments:
            label:string
        
        Returns:
            Relation object
        '''
        topicID = self.find_ID_by_label(label)
        # Randomly choose a ?predicate ?object pair
        desired_variables = ["predicate", "object"]
        query = f'''
        SELECT ?predicate ?object WHERE {{
            VALUES ?predicate {{ {self.sister_predicates} }}
            
            ?object rdfs:label ?label.
            wd:{topicID} ?predicate ?object.
            SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
        }}
        '''
        results = self.process_JSON(self.run_query(query), desired_variables)
        random_relation = self.random_items(results, count=1)[0]

        return question.Relation(random_relation[0], random_relation[1])
    def check_answer_is_correct_pattern_r(self, possible_answer, relation):
        return self.check_answer_is_correct_pattern(possible_answer, "wdt:" + relation.predicate, "wd:" + relation.object)
    def check_answer_is_correct_pattern(self, possible_answer, predicate, object):
        '''
        Description:
            Takes in possible answer and relation and generates a pattern to check if the answer is correct
            
        Arguments:
            possible_answer:string
            relation:Relation
            If variable ?v. If QID wd:QID. I can't append wd to things arbitrarily because I want this to handle variables or QIDs
            Similar for predicates and objects
        Returns:
            string - SPARQL query pattern
        '''
        # I don't like this system. There's probably a better way in SPARQL
        uniqueness_num = self.check_answer_is_correct_pattern_num_runs

        predicate_then_subclassstar = f"""{possible_answer} {predicate} ?checkAnswerIntermediate{uniqueness_num} .
                                            ?checkAnswerIntermediate{uniqueness_num} (wdt:P279)* {object} ."""
        # predicate is occupation. Check if possible_answer has a field of work practiced by object
        
        field_of_work_occupation = f"""#check if predicate is occupation
                                    FILTER({predicate} = {wikidata_properties['occupation']})
                                    {possible_answer} {wikidata_properties['field_of_work']} ?checkAnswerField{uniqueness_num} .
                                    ?checkAnswerField{uniqueness_num} {wikidata_properties["practiced by"]} {object} ."""
        # Append the patterns to a list
        patterns = []
        patterns.append(predicate_then_subclassstar)
        patterns.append(field_of_work_occupation)
        # UNION the patterns together
        # put curly braces around the patterns
        patterns = [f"{{{pattern}}}" for pattern in patterns]
        # join the patterns with a newline and a UNION
        patterns = "\nUNION\n".join(patterns)
        self.check_answer_is_correct_pattern_num_runs += 1
        return patterns
    def check_answer_using_pattern(self, possible_answer, relation):
        '''
        Description:
            Takes in possible answer and relation and checks if the answer is correct
        
        Arguments:
            possible_answer:string
            relation:Relation
        
        Returns:
            boolean
        '''
        patterns = self.check_answer_is_correct_pattern_r("wd:" + possible_answer, relation)
        query = f'''
        ASK WHERE {{
            {patterns}
        }}
        '''
        print(query)
        results = self.run_query(query)
        return results['boolean']
    
    def element_question_unified(self, QID, num_wrong_answers=3, num_questions=1):
        '''
        Description:
            Creates a question with the element as the correct answer
            Unlike element_question, it uses only one SPARQL query to find the relation and sister elements
        Arguments:
            element_label:string
            num_wrong_answers:int
        
        Returns:
            List of question:Question
        '''
        wrong_answer_variables = ["?wrong" + str(i) for i in range(num_wrong_answers)]
        wrong_answer_variables_str = " ".join(wrong_answer_variables)
        wrong_answer_filters = "\n".join([f"FILTER NOT EXISTS {{\n{self.check_answer_is_correct_pattern(wrong_answer_variable, "?predicate", "?object")}\n}}" for wrong_answer_variable in wrong_answer_variables])
        # Make sure wrong answers are unique. Requires n^2 queries
        wrong_uniqueness_filters = "\n".join([f"FILTER(?wrong{i} != ?wrong{j})" for i in range(num_wrong_answers) for j in range(i+1, num_wrong_answers)])
        #wrong_answers_share_predicate = "\n".join([self.check_answer_is_correct_pattern(wrong_answer_variable, "?sister_predicate", "?sister_item") for wrong_answer_variable in wrong_answer_variables])
        wrong_answers_share_predicate = "\n".join([f"{wrong_answer_variable} ?sister_predicate ?sister_item." for wrong_answer_variable in wrong_answer_variables])
        query = f'''
        SELECT DISTINCT ?predicate ?object {wrong_answer_variables_str} WHERE {{
            VALUES ?predicate {{ {self.sister_predicates} }}
            VALUES ?sister_predicate {{ {self.sister_predicates} }}
            FILTER(?predicate != ?sister_predicate) # Could make this more complicated so that each wrong answer can have its own sister_predicate
            {self.check_answer_is_correct_pattern("wd:" + QID, "?predicate", "?object")}
            wd:{QID} ?sister_predicate ?sister_item.
            #SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
            #FILTER(LANG(?object) = "en")
            FILTER(?object != wd:{QID})
            ?object rdfs:label ?label. # make sure the object has a label
            {wrong_answer_filters}
            {wrong_uniqueness_filters}
            {wrong_answers_share_predicate}
        }}
        LIMIT {num_questions}
        '''
        print(query)
        results = self.run_query(query)
        columns = ["predicate", "object"] + [v[1:] for v in wrong_answer_variables]
        print(columns)
        results = self.process_JSON(results, columns)
        
        questions = []
        for result in results:
            relation = question.Relation(result[0], result[1])
            wrong_answers = result[2:]
            questions.append(question.Question(relation, QID, wrong_answers))
        return questions
        
    def element_question(self, element_label, num_wrong_answers=3):
        '''
        Description:
           Creates a question with the element as the correct answer
        
        Arguments:
            element_label:string
            num_wrong_answers:int
        
        Returns:
            question:Question
        '''
        elementID = self.find_ID_by_label(element_label)
        
        relation = self.get_category_relation(element_label)
        categoryID = relation.object
        wrong_answers = self.sister_element(elementID, categoryID, [str(relation)])

        selected_answers = self.random_items(wrong_answers, count=num_wrong_answers)
        relation = question.Relation( self.find_label_by_ID(relation.predicate), self.find_label_by_ID(relation.object))
        return question.Question(relation, elementID, selected_answers)
    
    # Element(s) -> Categories Question Methods
    '''
    Which of the following is a category baseball, volleyball, basketball, and cricket all belong to?
    1. sport in ancient Greece
    2. *team sport*
    3. racing sports
    4. sport in Europe
    '''
    def find_category(self, elementIDs, predicateID=SUBCLASS_OF):
        '''
        Description:
            Inputs element IDs and outputs a category ID that they all belong to based on the predicate
        
        Arguments:
            elementIDs:list(string)
            predicateID:string
        
        Returns:
            random_categoryID:string -  random category that elementID belongs to
        '''
        # Prepare elements
        element_queries = "".join([f"wd:{e} " for e in elementIDs]) # wd:QXXXX for every element
        
        # Prepare query
        desired_variables = ["category"]
        query = f'''
        SELECT ?category ?label (COUNT(?elements) as ?elementCount) WHERE {{
            VALUES ?elements {{ {element_queries} }}
            ?elements wdt:{predicateID} ?category.
            ?category rdfs:label ?label.

            SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
            FILTER(LANG(?label) = "en")
        }}
        GROUP BY ?category ?label
        HAVING (COUNT(?elements) = {len(elementIDs)})
        '''
           
        categories = self.process_JSON(self.run_query(query), desired_variables)
        if len(categories) == 0:
            print(query)
            raise Exception("These elements do not have a shared category")
        categories = [c[0] for c in categories]
        random_categoryID = self.random_items(categories, count=1)[0]
        
        return random_categoryID
    def category_question(self, element_labels, num_wrong_answers=3):
        '''
        Description:
            Inputs elements and outputs 4 categories, one being the one that contains every element
        
        Arguments:
            element_labels:list(string)
            num_wrong_answers:int
        
        Returns:
            question:Question
        '''
        # Get category of the elements
        elementIDs = []
        for element_label in element_labels:
            elementIDs.append(self.find_ID_by_label(element_label))
        categoryID = self.find_category(elementIDs) # Default is SUBCLASS_OF
        
        # Get 3 other sister categories that are similar, but not the correct answer
        print(elementIDs)
        results, _ = self.sister_category(categoryID, elementIDs, n_items=num_wrong_answers) 

        # Prepare object of relation
        object = ""
        i=0
        while i < len(element_labels) - 1:
            object += f"{element_labels[i]}, "
            i+=1
        
        if len(element_labels) == 1:
            object += f"{element_labels[i]} belongs to"
        else:
            object += f"and {element_labels[i]} all belong to"

        # Return the 3 sisters and the correct category
        relation = question.Relation("category", object)
        
        return question.Question(relation, categoryID, results)


In [75]:
# print("What element do you want the correct answer to the question to be?", end=": ")
# element = input()
# print(element)
# print("How many questions do you want?", end=": ")
# num_questions = int(input())
# print(num_questions)

In [76]:
named_after = "wdt:P138"
occupation = "wdt:P106"
has_use = "wdt:P366"
studied_in = "wdt:P2579"
significant_event = "wdt:P793"
member_of_political_party = "wdt:P102"
sensible_sister_predicates = f"wdt:P279 wdt:P101 wdt:P361 wdt:P921 wdt:P131 wdt:P150 {named_after} {occupation} {has_use} {studied_in} {significant_event} {member_of_political_party}"

generator = Generator(sister_predicates=sensible_sister_predicates)

In [63]:
import time
def question_answer(question):
    generator.print_question(question)
    time.sleep(0.2)

    answer = int(input("What is your answer?")) - 1
    user_answer = generator.find_label_by_ID(question.all_answers[answer])
    print(f"Your answer was {user_answer}")
    correct_answer = generator.find_label_by_ID(question.correct_answer)
    print(f"The correct answer was {correct_answer}")
    if user_answer == correct_answer:
        print("You are correct!")
    else:
        print("You are incorrect!")

In [64]:
cat_q = generator.category_question(["baseball", "volleyball", "basketball", "cricket"])
question_answer(cat_q)

['Q5369', 'Q1734', 'Q5372', 'Q5375']
Which of the following is a category baseball, volleyball, basketball, and cricket all belong to?
['Q3493923', 'Q3493900', 'Q216048', 'Q3493920']
1. sport in Europe
2. outdoor sports
3. team sport
4. sport in ancient Greece


ValueError: invalid literal for int() with base 10: ''

In [82]:
check_answer_using_pattern = generator.check_answer_using_pattern

In [23]:
politician = "Q82955"
occupation = "P106"
does_politics_relation = question.Relation(occupation, politician)
print(str(does_politics_relation))

wdt:P106 wd:Q82955


In [50]:
charles_de_gaulle = "Q2042"
ahn = "Q20995"
cdg_politics = generator.check_answer_using_pattern(ahn, does_politics_relation)
print(cdg_politics)


        ASK WHERE {
            {wd:Q20995 wdt:P106 ?checkAnswerIntermediate3 .
                                            ?checkAnswerIntermediate3 (wdt:P279)* wd:Q82955 .}
UNION
{#check if predicate is occupation
                                    FILTER(wdt:P106 = wdt:P106)
                                    wd:Q20995 wdt:P101 ?checkAnswerField3 .
                                    ?checkAnswerField3 wdt:P3095 wd:Q82955 .}
        }
        
True


In [77]:
testLabel = "Joe Biden"
testQID = generator.find_ID_by_label(testLabel)

In [81]:

my_question = generator.element_question_unified(testQID, 3, 1)
print("\n".join([str(q) for q in my_question]))


        SELECT DISTINCT ?predicate ?object ?wrong0 ?wrong1 ?wrong2 WHERE {
            VALUES ?predicate { wdt:P279 wdt:P101 wdt:P361 wdt:P921 wdt:P131 wdt:P150 wdt:P138 wdt:P106 wdt:P366 wdt:P2579 wdt:P793 wdt:P102 }
            VALUES ?sister_predicate { wdt:P279 wdt:P101 wdt:P361 wdt:P921 wdt:P131 wdt:P150 wdt:P138 wdt:P106 wdt:P366 wdt:P2579 wdt:P793 wdt:P102 }
            FILTER(?predicate != ?sister_predicate) # Could make this more complicated so that each wrong answer can have its own sister_predicate
            {wd:Q6279 ?predicate ?checkAnswerIntermediate9 .
                                            ?checkAnswerIntermediate9 (wdt:P279)* ?object .}
UNION
{#check if predicate is occupation
                                    FILTER(?predicate = wdt:P106)
                                    wd:Q6279 wdt:P101 ?checkAnswerField9 .
                                    ?checkAnswerField9 wdt:P3095 ?object .}
            wd:Q6279 ?sister_predicate ?sister_item.
            #SERVIC

EndPointInternalError: EndPointInternalError: The endpoint returned the HTTP status code 500. 

Response:
b'SPARQL-QUERY: queryStr=\n        SELECT DISTINCT ?predicate ?object ?wrong0 ?wrong1 ?wrong2 WHERE {\n            VALUES ?predicate { wdt:P279 wdt:P101 wdt:P361 wdt:P921 wdt:P131 wdt:P150 wdt:P138 wdt:P106 wdt:P366 wdt:P2579 wdt:P793 wdt:P102 }\n            VALUES ?sister_predicate { wdt:P279 wdt:P101 wdt:P361 wdt:P921 wdt:P131 wdt:P150 wdt:P138 wdt:P106 wdt:P366 wdt:P2579 wdt:P793 wdt:P102 }\n            FILTER(?predicate != ?sister_predicate) # Could make this more complicated so that each wrong answer can have its own sister_predicate\n            {wd:Q6279 ?predicate ?checkAnswerIntermediate9 .\n                                            ?checkAnswerIntermediate9 (wdt:P279)* ?object .}\nUNION\n{#check if predicate is occupation\n                                    FILTER(?predicate = wdt:P106)\n                                    wd:Q6279 wdt:P101 ?checkAnswerField9 .\n      

In [80]:
for q in my_question:
    question_answer(q)

Which of the following is a member of political party Democratic Party?
['Q157', 'Q329', 'Q6279']
1. François Hollande
2. Nicolas Sarkozy
3. Joe Biden
Your answer was Joe Biden
The correct answer was Joe Biden
You are correct!
Which of the following is a member of political party Democratic Party?
['Q352', 'Q6279', 'Q157']
1. Adolf Hitler
2. Joe Biden
3. François Hollande
Your answer was Joe Biden
The correct answer was Joe Biden
You are correct!
Which of the following is a member of political party Democratic Party?
['Q6279', 'Q157', 'Q475']
1. Joe Biden
2. François Hollande
3. Eduardo Frei Montalva
Your answer was Joe Biden
The correct answer was Joe Biden
You are correct!
Which of the following is a member of political party Democratic Party?
['Q498', 'Q6279', 'Q157']
1. Ulrich Frédéric Woldemar, Comte de Lowendal
2. Joe Biden
3. François Hollande
Your answer was Joe Biden
The correct answer was Joe Biden
You are correct!
Which of the following is a member of political party Democra

In [97]:
my_question = generator.element_question("net neutrality", 3)
question_answer(my_question)

Which of the following is a subclass of principle?
['Q1145689', 'Q2135371', 'Q25004657', 'Q853467']
1. Hindu law
2. Swiss law
3. right-to-try law
4. net neutrality
Your answer was net neutrality
The correct answer was net neutrality
You are correct!


In [ ]:

def get_wikidata_item(item_id):
    # Wikidata API endpoint
    url = "https://www.wikidata.org/w/api.php"
    
    # Parameters for the API request
    params = {
        "action": "wbgetentities",  # Action to get data about entities
        "ids": item_id,  # ID of the Wikidata item (e.g., Q64 for Berlin)
        "format": "json",  # Response format
        "props": "labels|descriptions|aliases",  # Properties to retrieve: labels, descriptions, and aliases
        "languages": "en"  # Language filter
    }
    
    # Make the GET request to the Wikidata API
    response = requests.get(url, params=params)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        
        # Access the item's data
        item_data = data['entities'][item_id]
        
        # Extract and print the label, description, and aliases
        label = item_data['labels']['en']['value']
        description = item_data['descriptions']['en']['value']
        aliases = [alias['value'] for alias in item_data['aliases']['en']]
        
        print(f"Label: {label}")
        print(f"Description: {description}")
        print(f"Aliases: {', '.join(aliases)}")
    else:
        print("Failed to retrieve data")

# Example usage
get_wikidata_item("Q64")